<a href="https://colab.research.google.com/github/vicknentura/tripleten/blob/main/MercedesGLC300Re_entryProgram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, MinMaxScaler

In [14]:
data=pd.read_excel("Revised Budget.xlsx", sheet_name="Sheet6")

current_year = 2024
data['Age'] = current_year - data['Year']

# Invert 'Previous Owners' to reflect negative impact
max_owners = data['Previous Owners'].max()
data['Inverted Owners'] = max_owners - data['Previous Owners']

# Ordinal encoding for 'Damage History'
damage_categories = ['No', 'Minor', 'Yes', 'Major']  # Define order
ordinal_encoder = OrdinalEncoder(categories=[damage_categories])
data['Damage History Encoded'] = ordinal_encoder.fit_transform(data[['Damage History']])

# Use the new features in the model
X = data[['Mileage', 'Age', 'Inverted Owners', 'Damage History Encoded']]
y = data['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

print(f"Intercept: {model.intercept_}")
print(f"Mileage Coefficient: {model.coef_[0]}")
print(f"Age Coefficient: {model.coef_[1]}")

Mean Squared Error: 2603261.782136588
R-squared: 0.3836022197768658
Intercept: 26946.309944781802
Mileage Coefficient: -0.05906245929924383
Age Coefficient: -903.8146239492039


In [15]:
# 1. Normalize relevant columns
scaler = MinMaxScaler()
data[['Year_Scaled', 'Mileage_Scaled', 'Price_Scaled', 'Owners_Scaled']] = scaler.fit_transform(
    data[['Year', 'Mileage', 'Price', 'Previous Owners']]
)

# 2. Map 'Damage History' to numerical values
damage_mapping = {'No': 0, 'Minor': 1, 'Yes': 2, 'Major': 3}
data['Damage_Numeric'] = data['Damage History'].map(damage_mapping)

# 3. Assign weights (adjust as needed)
year_weight = 0.3
mileage_weight = 0.2
price_weight = 0.2
owners_weight = 0.2
damage_weight = 0.1

# 4. Calculate composite score
data['Composite_Score'] = (
    data['Year_Scaled'] * year_weight
    + (1 - data['Mileage_Scaled']) * mileage_weight  # Invert mileage for lower is better
    + (1 - data['Price_Scaled']) * price_weight  # Invert price for lower is better
    + (1 - data['Owners_Scaled']) * owners_weight  # Invert owners for lower is better
    + (1 - data['Damage_Numeric'] / 3) * damage_weight  # Normalize and invert damage for lower is better
)

# 5. Find the row with the highest score
best_score_index = data['Composite_Score'].idxmax()
best_score_row = data.loc[best_score_index]

print("\nRow with the best score:")
print(best_score_row)


Row with the best score:
Year                          2020
Mileage                      93250
Price                        18000
Previous Owners                  1
Damage History                  No
Age                              4
Inverted Owners                  3
Damage History Encoded         0.0
Year_Scaled               0.666667
Mileage_Scaled            0.527143
Price_Scaled              0.733333
Owners_Scaled                  0.0
Damage_Numeric                   0
Composite_Score           0.647905
Name: 12, dtype: object


In [16]:
top_five = data.sort_values(by=['Composite_Score'], ascending=False).head(5)

print("\nTop five ranking:")
print(top_five)


Top five ranking:
    Year  Mileage  Price  Previous Owners Damage History  Age  \
12  2020    93250  18000                1             No    4   
13  2019   102000  12800                2          Minor    5   
18  2019    68400  19000                1             No    5   
28  2019    59000  19900                1             No    5   
36  2019   107110  16500                1             No    5   

    Inverted Owners  Damage History Encoded  Year_Scaled  Mileage_Scaled  \
12                3                     0.0     0.666667        0.527143   
13                2                     1.0     0.500000        0.610476   
18                3                     0.0     0.500000        0.290476   
28                3                     0.0     0.500000        0.200952   
36                3                     0.0     0.500000        0.659143   

    Price_Scaled  Owners_Scaled  Damage_Numeric  Composite_Score  
12      0.733333       0.000000               0         0.647905  

In [17]:
# 1. Normalize relevant columns
scaler = MinMaxScaler()
data[['Year_Scaled', 'Mileage_Scaled', 'Price_Scaled', 'Owners_Scaled']] = scaler.fit_transform(
    data[['Year', 'Mileage', 'Price', 'Previous Owners']]
)

# 2. Map 'Damage History' to numerical values
damage_mapping = {'No': 0, 'Minor': 1, 'Yes': 2, 'Major': 3}
data['Damage_Numeric'] = data['Damage History'].map(damage_mapping)

# 3. Assign weights (adjust as needed)
year_weight = 0.3
mileage_weight = 0.2
price_weight = 0.2
owners_weight = 0.2
damage_weight = 0.1

# 4. Calculate composite score
data['Composite_Score'] = (
    data['Year_Scaled'] * year_weight
    + (1 - data['Mileage_Scaled']) * mileage_weight  # Invert mileage for lower is better
    + (1 - data['Price_Scaled']) * price_weight  # Invert price for lower is better
    + (1 - data['Owners_Scaled']) * owners_weight  # Invert owners for lower is better
    + (1 - data['Damage_Numeric'] / 3) * damage_weight  # Normalize and invert damage for lower is better
)

# 5. Find the row with the highest score
best_score_index = data['Composite_Score'].idxmax()
best_score_row = data.loc[best_score_index]

print("\nRow with the best score:")
print(best_score_row)

# 6. Get the top five rows based on Composite Score
top_five = data.sort_values(by=['Composite_Score'], ascending=False).head(5)

print("\nTop five ranking:")
print(top_five)


Row with the best score:
Year                          2020
Mileage                      93250
Price                        18000
Previous Owners                  1
Damage History                  No
Age                              4
Inverted Owners                  3
Damage History Encoded         0.0
Year_Scaled               0.666667
Mileage_Scaled            0.527143
Price_Scaled              0.733333
Owners_Scaled                  0.0
Damage_Numeric                   0
Composite_Score           0.647905
Name: 12, dtype: object

Top five ranking:
    Year  Mileage  Price  Previous Owners Damage History  Age  \
12  2020    93250  18000                1             No    4   
13  2019   102000  12800                2          Minor    5   
18  2019    68400  19000                1             No    5   
28  2019    59000  19900                1             No    5   
36  2019   107110  16500                1             No    5   

    Inverted Owners  Damage History Encoded  Year_